### Create pickles having the needed features
This is first time I'm doing things like this. It was unclear, based on the original study, in what phase all the pickles were made?
I'm making them in this file.

Lot of debugging included...

In [1]:
# Feature_Creation.ipynb
# Imported libraries
import datetime
import dask.array as da
import zarr
import numcodecs
import datetime
from datetime import datetime
import dask
from dask import delayed, compute
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import numpy as np
import pandas as pd
from scipy.stats import skew
from skimage.filters import gabor
from sklearn.impute import KNNImputer
from scipy.ndimage import generic_filter as gf
from dask_image.ndfilters import generic_filter as d_gf
from scipy.ndimage import gaussian_filter, sobel
import pickle
# Import the required functions from feature_creation.py in Functions directory
from Functions import feature_creation
from Functions import feature_creation2
from Functions import general_functions
#from Functions import post_processing
from Functions.general_functions import create_circular_mask
import os

from numba import config
config.DEBUG = True

In [2]:
import importlib
import Functions.feature_creation2
importlib.reload(Functions.feature_creation)


<module 'Functions.feature_creation' from 'e:\\Gradu\\PurOja\\02_Analysis\\01_Scripts\\Functions\\feature_creation.py'>

In [2]:
# Paths to feature files
features_path = "../../01_Data/01_Raw/features/features" 
output_path = "../../01_Data/01_Raw/features"

In [3]:
# Set up a Dask cluster with adaptive scaling
cluster = LocalCluster(
    processes=True,                # Use separate processes for parallelism
    n_workers=10,                  # Match with available CPU cores
    threads_per_worker=1,          # One thread per worker
    memory_limit="20GB"            # Memory limit per worker
)

# Enable adaptive scaling
cluster.adapt(minimum=1, maximum=15)  # Adjust workers dynamically (1 to 15 workers)

# Connect the client to the cluster
client = Client(cluster)

# Configure memory management
dask.config.set({
    'distributed.worker.memory.target': 0.7,  # Spill memory starts at 70% usage
    'distributed.worker.memory.spill': 0.8,  # Spill to disk when 80% memory is used
    'distributed.worker.memory.pause': 0.95, # Pause computation at 95% memory usage
})

# Check cluster dashboard link
print("Dask Dashboard:", client.dashboard_link)


Dask Dashboard: http://127.0.0.1:8787/status


Separate pickle creation was kept in here to test the data.

In [4]:
def d_gf(array, func, footprint):
        """
        Wrapper function to safely apply a mask or function with Dask arrays.
        Handles edge cases for padding and ensures indices are valid.
        """
        depth = footprint.shape[0] // 2
        depth = max(0, depth)  # Ensure depth is non-negative

        # Apply the mask with proper overlap and safe boundary handling
        return da.map_overlap(
        func, array, depth=depth, boundary="reflect", trim=True
    )
        # Helper function to add feature columns conditionally


def load_raw(zone_number, chunk_size=(1666, 1666)):
        global features_path, output_path
        data_frame = None  # Ensure data_frame is explicitly initialized

        try:
            # Load raw feature files
            labels = np.load(f"{features_path}/Label3m_{zone_number}.npy")
            hpmf = np.load(f"{features_path}/HPMF_{zone_number}.npy")
            skyview = np.load(f"{features_path}/SVF_{zone_number}.npy").astype(np.float64)
            impoundment = np.load(f"{features_path}/Impoundment_{zone_number}.npy")
            slope = np.load(f"{features_path}/slope_{zone_number}.npy")
            DEM = np.load(f"{features_path}/DEM_{zone_number}.npy")
            # Replace -99999 with np.nan in DEM using Dask
            dask_dem = da.from_array(DEM, chunks=chunk_size)
            dask_dem = da.where(dask_dem == -99999, np.nan, dask_dem)

            # Trigger computation and persist changes
            dask_dem = dask_dem.persist()  # Persist in memory for future use

            print(f"Loaded files for zone {zone_number}:")
            print(f"Labels shape: {labels.shape}")
            print(f"HPMF shape: {hpmf.shape}")
            print(f"Skyview shape: {skyview.shape}")
            print(f"Impoundment shape: {impoundment.shape}")
            print(f"Slope shape: {slope.shape}")
            print(f"DEM shape: {DEM.shape}")

            # Assertions to validate inputs
            assert labels is not None, "Error: 'labels' is not defined or is None."
            assert hpmf is not None, "Error: 'hpmf' is not defined or is None."
            assert skyview is not None, "Error: 'skyview' is not defined or is None."

            # Feature creation
            stream_amp = feature_creation2.stream_amplification(impoundment)

            data_frame = pd.DataFrame({
                "label_3m": labels.reshape(-1),
                "hpmf_raw": hpmf.reshape(-1),
                "skyview_raw": skyview.reshape(-1),
                "impoundment_raw": impoundment.reshape(-1),
                "slope_raw": slope.reshape(-1),
            })

            # Add additional features (e.g., DEM)
            if "DEM_raw" not in data_frame.columns:
                data_frame["DEM_raw"] = DEM.flatten()

        except FileNotFoundError as e:
            print(f"FileNotFoundError: {e}")
            return None
        except AssertionError as e:
            print(f"AssertionError: {e}")
            return None
        except Exception as e:
            print(f"General Error: {e}")
            return None

        # Final fallback check
        if data_frame is None:
            raise RuntimeError("Failed to create or load 'data_frame'. Check input data and processing steps.")


    #------------- MANUAL ----------------
    #------------- Dem -------------------
    # Add DEM feature if not already in DataFrame
        def add_features(data_frame, DEM, skyview, impoundment, hpmf, slope, zone_number):
                if "DEM_raw" not in data_frame.columns:  
                    data_frame["DEM_raw"] = dask_dem.compute().flatten() # -99999 values to Nan

                    print("Defining tasks...")

        #------------- Helper function adds features to the data_frame -------------------

        # Add features derived from DEM
        print("Processing DEM features...", datetime.now())
        # Ensure the shape of the DEM array matches expected dimensions
        dem_shape = dask_dem.shape
        assert dem_shape == (5000, 5000), f"Unexpected DEM shape: {dem_shape}"
        if "DEM_ditch_detection" not in data_frame.columns:
            data_frame["DEM_ditch_detection"] = feature_creation2.dem_ditch_detection(dask_dem.compute()).reshape(-1) # -99999 to Nan

        # I'm interested in all the streams
        if "DEM_ditch_detection_streams" not in data_frame.columns:
            data_frame["DEM_ditch_detection_streams"] = feature_creation2.impoundment_DEM_stream_strenghten(data_frame["DEM_ditch_detection"].values.reshape((5000,5000)), stream_amp).reshape(-1)
            print("Added DEM ditches detected.")

                    # I'm interested in all the streams
                    #------------- Sky view factor -------------------
                    # Add features derived from Skyview
            #conic means from sky view factor
        print("Processing Conic Mean features...", datetime.now())
        if "conic_mean" not in data_frame.columns:
            print ("Calculating conic_mean...")
            count_conic_mean = feature_creation2.conic_mean(skyview, 6, 0.9894350171089172)
            data_frame["conic_mean"] = count_conic_mean.flatten()
            print("Added conic_mean to the DataFrame.")
        #skyview gabor
        print("Processing SkyView Gabor features...", datetime.now())
        if "skyview_gabor" not in data_frame:
            print ("Calculating skyview_gabor...")
            data_frame["skyview_gabor"] = feature_creation2.sky_view_gabor(skyview).reshape(-1)
            print("Added skyview_gabor to the DataFrame.")
        
        #if "skyview_ditch" not in data_frame.columns:
        #    print ("Calculating skyview ditch...")
        #    data_frame["skyview_ditch"] = feature_creation2._reclassify_sky_view_non_ditch_amp(skyview).reshape(-1)
        #    print("Added skyview_ditch to the DataFrame.")

        print("starting-impoundment:    ", datetime.now())

        print("Processing Impoundment features...")
        if "impoundment_amplified" not in data_frame.columns:
            print("Calculating impoundment_amplified...")
            data_frame["impoundment_amplified"] = feature_creation2.impoundment_amplification(impoundment).reshape(-1)
            print("Added 'impoundment_amplified' to the DataFrame.")

        print("starting-hpmf manual:    ", datetime.now())

        if "hpmf_f" not in data_frame.columns:
            print("Calculating hpmf_f...")
            count_hpmf_f = feature_creation2.process_hpmf(hpmf, 5)
            data_frame["hpmf_f"] = count_hpmf_f.flatten()
            print("Added 'hpmf_f' to the DataFrame.")
        
        if "hpmf_f_visualisation" not in data_frame.columns: 
            print("Calculating hpmf_f_visualisation...")
            count_hpmf_fv = feature_creation2.hpmf_f_visualisation(hpmf, 6) 
            data_frame["hpmf_f_visualisation"] = count_hpmf_fv.flatten() 
            print("Added 'hpmf_f_visualisation' to the DataFrame.")

        print("starting-slope manual:    ", datetime.now())
        if "slope_channels" not in data_frame:
            print("Calculating slope_channels...")
            slope_processed_dask = feature_creation2.process_slope_for_channels(slope, chunk_size=chunk_size)
            data_frame["slope_channels"] = slope_processed_dask.compute().reshape(-1) 
            print("Added 'slope_channels' to the DataFrame.")
            
            print(f"Feature generation completed for zone {zone_number}.")
                    
        # Return the final DataFrame once features are added
            #Final fallback check for undefined data_frame
            if 'data_frame' not in locals():
                raise RuntimeError("Failed to create or load 'data_frame'. Please check input data and pickle files.")
     
            return data_frame
#------------- AUTOMATED --------------------------
#------------- Process dataframe-------------------
def process_dataframe(data_frame, skyview, impoundment, hpmf, slope, chunk_size):
    print("Starting feature computation: ", datetime.now())

    def add_feature(column_name_base, compute_func, array, radii, *args):
        """
        Add a feature to the data frame, ensuring consistent naming.
        """
        for radius in radii:
            column_name = f"{column_name_base}_{radius}"  # Construct consistent column names
            if column_name not in data_frame.columns:
                print(f"Generating feature: {column_name}")
                data_frame[column_name] = compute_func(array, *args).flatten()

        print(f"Feature generation completed for zone {zone_number}.")
        print(f"Shape of feature array before flattening: {array.shape}")
        print(f"Shape after computation: {compute_func(array, *args).shape}")

    # Handling NaN values for np.amax (replace NaNs with -infinity)
    skyview_no_nan = np.nan_to_num(skyview, nan=-np.inf)

    # List of features to generate
    features = [
        ("skyview", skyview, [2, 4, 6], np.nanmax, "max"),
        ("skyview", skyview, [6], np.nanmin, "min"),
        ("skyview", skyview, [2, 4, 6], np.nanmedian, "median"),
        ("skyview", skyview, [6], np.nanstd, "std"),
        ("skyview", skyview, [2], skew, "skew"),
        ("impoundment", impoundment, [3, 2, 6], np.nanmean, "mean"),
        ("impoundment", impoundment, [2, 4, 6], np.nanmedian, "median"),
        ("impoundment", impoundment, [6], np.amax, "max"),
        ("impoundment", impoundment, [4, 6], np.nanstd, "std"),
        ("impoundment", impoundment, [2], skew, "skew"),
        ("hpmf", hpmf, [2, 3, 4, 6], np.nanmean, "mean"),
        ("hpmf", hpmf, [3, 4], np.amin, "min"),
        ("hpmf", hpmf, [4], np.nanmedian, "median"),
        ("hpmf", hpmf, [6], np.nanstd, "std"),
        ("hpmf", hpmf, [2], skew, "skew"),
        ("slope", slope, [6], np.nanmedian, "median"),
        ("slope", slope, [2, 4, 6], np.amin, "min"),
        ("slope", slope, [4, 6], np.nanstd, "std"),
        ("slope", slope, [2], skew, "skew"),
    ]

    # Loop through features to calculate them
    for col_name, array, feature_sizes, func, func_name in features:
        # Check if the column already exists
        if col_name not in data_frame.columns:
            print(f"Generating feature: {col_name}")

            # Apply the function to generate the feature
            result = func(array)

            # Check if result is a scalar and handle accordingly
            if np.isscalar(result):  # If it's a scalar
                data_frame[col_name] = np.full(len(data_frame), result)  # Fill the column with the scalar value
            else:
                # Ensure the result is a NumPy array and flatten it
                if not isinstance(result, np.ndarray):
                    result = np.array(result)  # Convert to numpy array if necessary
                data_frame[col_name] = result.flatten()  # Flatten the array and assign to DataFrame

            print(f"Added column: {col_name}")

#-----------------------------------------------------------------------
    # Feature computation for regular features
    for col_name, array, feature_sizes, func, func_name in features:
        if col_name not in data_frame.columns:
            print(f"Generating feature: {col_name}")
            
            # If the feature requires a circular mask and chunked computation
            if func in [np.amax, np.nanmean, np.nanstd, np.nanmedian, skew]:  # Add the relevant functions
                for radius in feature_sizes:
                    col_name_with_radius = f"{col_name}_{radius}"
                    if col_name_with_radius not in data_frame.columns:
                        print(f"Generating feature: {col_name_with_radius}")
                        result = d_gf(da.from_array(array, chunks=chunk_size), func, footprint=general_functions.create_circular_mask(radius)).compute(scheduler='processes').reshape(-1)
                        data_frame[col_name_with_radius] = result.flatten()
                        print(f"Added column: {col_name_with_radius}")
            else:
                result = func(array)
                if np.isscalar(result):
                    result = np.full(len(data_frame), result)
                data_frame[col_name] = result.flatten()
                print(f"Added column: {col_name}")

    # Handling scalar features with radius
    for prefix, array, radii, func, suffix in features:
        for radius in radii:
            col_name = f"{prefix}_{suffix}_{radius}"
            if col_name not in data_frame.columns:
                print(f"Generating feature: {col_name}")

                # Compute the feature (mean, std, etc.)
                feature = func(array)

                # If the result is a scalar, broadcast it to match the DataFrame length
                if np.isscalar(feature):
                    feature = np.full(len(data_frame), feature)

                # Add the feature to the DataFrame
                data_frame[col_name] = feature.flatten()
                print(f"Added column: {col_name}")

                # Apply circular mask (if needed)
                mask = general_functions.create_circular_mask(radius)
                array_2d = array.reshape((5000, 5000))  # Assuming 5000x5000 shape (adjust accordingly)
                array_dask = da.from_array(array_2d, chunks=chunk_size)

                # Define the function to apply mask to each block
                def apply_mask_to_block(block, mask):
                    """
                    Applies a mask to the given block.
                    Pads the mask if necessary to match the block size.
                    """
                    block_height, block_width = block.shape
                    mask_height, mask_width = mask.shape

                    # Pad the mask to fit the block dimensions
                    padded_mask = np.pad(
                        mask, 
                        ((0, max(0, block_height - mask_height)), (0, max(0, block_width - mask_width))),
                        mode='constant', 
                        constant_values=0
                    )
                    return block * padded_mask[:block_height, :block_width]

                # Apply the masking function to each block

                    masked_array = array_dask.map_blocks(
                        apply_mask_to_block,
                        mask=mask,  # Pass the precomputed mask as a keyword argument
                        dtype=np.float32  # Explicitly specify the dtype
                    ).compute()

                print("Applied mask to the array.")
                print(f"Applied mask for {col_name} with radius {radius}")


    print("Feature computation completed: ", datetime.now())
    return data_frame

#------------- Ordering dataframe-------------------
def set_order(data_frame):
    """
    Reorder columns for final DataFrame, checking for missing columns.
    """
    ordered_columns = ordered_columns = [
        'label_3m', 
		'hpmf_raw', 
		'skyview_raw', 
		'impoundment_raw', 
		'slope_raw', 
		'DEM_ditch_detection', 
		'DEM_ditch_detection_streams',
		'conic_mean',
		'skyview_gabor', 
		'skyview_min_6', 
		'skyview_median_2', 
		'skyview_median_6', 
		'skyview_max_2', 
		'skyview_max_4', 
		'skyview_max_6',
		'skyview_std_6', 
		'impoundment_amplified', 
		'impoundment_mean_2', 
        'impoundment_mean_3', 
		'impoundment_mean_4',
		'impoundment_mean_6',
		'impoundment_median_2', 		
		'impoundment_median_4', 
 		'impoundment_median_6', 
		'impoundment_max_6', 
        'impoundment_std_4', 
		'impoundment_std_6',
        'impoundment_std_6hpmf_f',
		'hpmf_f', 
        'hpmf_f_visualisation',
        'hpmf_filter',
		'hpmf_gabor', 		
		'hpmf_min_2', 
        'hpmf_min_4',
		'hpmf_mean_3', 		
		'hpmf_mean_6', 
		'hpmf_mean_4',
        'hpmf_median_4', 
		'hpmf_std_6', 
		'slope_channels',
        'slope_min_2', 	
		'slope_min_4', 		
		'slope_min_6',
		'slope_median_6',
		'slope_mean_6',
		'slope_std_4', 
		'slope_std_6', 
    ]

    # Check the columns of the data frame before reordering
    print("Generated columns in data_frame:", data_frame.columns)

    # Check if all ordered columns exist in the DataFrame
    existing_columns = [col for col in ordered_columns if col in data_frame.columns]

    # Log any missing columns
    missing_columns = set(ordered_columns) - set(existing_columns)
    if missing_columns:
        print(f"Warning: The following columns are missing and will be excluded: {missing_columns}")

    # If there are missing columns, we will return the data frame with the available columns only
    ordered_columns_with_existing = [col for col in ordered_columns if col in existing_columns]

    # Return the DataFrame with only the available columns in the desired order
    return data_frame[ordered_columns_with_existing]

# Define the range of zone numbers
zone_numbers = range(7, 11)  # Set the zone number range
chunk_size = (1666, 1666)  # This is basically size 5000/3, it's close enough to keep the shape

# Loop through the zone numbers
for zone_number in zone_numbers:
    # Define the path to the pickle file
    pickle_file_path = f"{output_path}/zone_{zone_number}.pickle"
    
    # Delete the existing pickle file if it exists (to ensure a fresh start)
    if os.path.exists(pickle_file_path):
        try:
            os.remove(pickle_file_path)
            print(f"Deleted corrupted or incomplete pickle file for zone {zone_number}. Starting fresh.")
        except Exception as e:
            print(f"Error deleting file for zone {zone_number}: {e}")

    # Load raw data
    zone_with_features = load_raw(zone_number, chunk_size)
    print(zone_with_features)  # This should print the DataFrame or an informative message

    if zone_with_features is None:
        raise ValueError("Failed to load raw data; 'zone_with_features' is None")

    # Check the shape and dimensions of the skyview data
    skyview_data = zone_with_features["skyview_raw"].values

    print("Shape of skyview_raw:", skyview_data.shape)
    print("Number of dimensions of skyview_raw:", skyview_data.ndim)

    # Conditional check for dimensionality
    if skyview_data.ndim == 1:
        print("skyview_raw is 1D")
    elif skyview_data.ndim == 2:
        print("skyview_raw is 2D")
    elif skyview_data.ndim == 3:
        print("skyview_raw is 3D")
    else:
        print("skyview_raw has more than 3 dimensions")

    # Process the dataframe
    zone_with_features = process_dataframe(
        zone_with_features,
        skyview=zone_with_features["skyview_raw"].values,
        impoundment=zone_with_features["impoundment_raw"].values,
        hpmf=zone_with_features["hpmf_raw"].values,
        slope=zone_with_features["slope_raw"].values,
        chunk_size=chunk_size
    )

    # Set the order for the features
    zone_with_features = set_order(zone_with_features)

    # **Save the processed features to a pickle file inside the loop**
    with open(pickle_file_path, "wb") as pickle_file:
        pickle.dump(zone_with_features, pickle_file)

    print(f"Zone {zone_number} features saved to pickle at {pickle_file_path}.")

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 190.74 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Loaded files for zone 7:
Labels shape: (5000, 5000)
HPMF shape: (5000, 5000)
Skyview shape: (5000, 5000)
Impoundment shape: (5000, 5000)
Slope shape: (5000, 5000)
DEM shape: (5000, 5000)


2025-03-30 10:16:17,331 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:45481'.
2025-03-30 10:56:16,821 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:39711'.
2025-03-30 10:56:19,722 - distributed.nanny - WARNING - Worker process still alive after 3.199999694824219 seconds, killing
2025-03-30 10:58:35,180 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:45765'.
2025-03-30 10:58:37,970 - distributed.nanny - WARNING - Worker process still alive after 3.1999989318847657 seconds, killing
2025-03-30 11:39:27,865 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:38251'.
2025-03-30 11:39:30,481 - distributed.nanny - WARNING - Worker process still alive after 3.1999995422363288 seconds, killing
2025-03-30 11:41:23,502 - tornado.application - ERROR - Exception in callback functools.partial(<function TCPS

2025-03-30 10:16:17,343 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2025-03-30 10:16:19,070 - distributed.core - INFO - Connection to tcp://127.0.0.1:36645 has been closed.
2025-03-30 10:16:21,072 - distributed.nanny - ERROR - Worker process died unexpectedly


-------------------------IR DUMP: create_circular_mask--------------------------
label 0:
    radius = arg(0, name=radius)             ['radius']
    $const2.0 = const(int, 2)                ['$const2.0']
    $6binary_multiply.2 = $const2.0 * radius ['$6binary_multiply.2', '$const2.0', 'radius']
    $const8.3 = const(int, 1)                ['$const8.3']
    size = $6binary_multiply.2 + $const8.3   ['$6binary_multiply.2', '$const8.3', 'size']
    $14load_global.5 = global(np: <module 'numpy' from '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/numpy/__init__.py'>) ['$14load_global.5']
    $16load_attr.6 = getattr(value=$14load_global.5, attr=zeros) ['$14load_global.5', '$16load_attr.6']
    $22build_tuple.9 = build_tuple(items=[Var(size, general_functions.py:72), Var(size, general_functions.py:72)]) ['$22build_tuple.9', 'size', 'size']
    $24load_global.10 = global(np: <module 'numpy' from '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/num

...

Added 'slope_channels' to the DataFrame.
Feature generation completed for zone 10.


2025-03-30 13:35:19,150 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.


          label_3m  hpmf_raw  skyview_raw  impoundment_raw    slope_raw  \
0                0  0.110000     0.983332              0.0     6.923639   
1                0  0.115000     0.982089              0.0     7.535480   
2                0  0.121406     0.981108              0.0     8.515753   
3                0  0.129219     0.980160              0.0     9.888376   
4                0  0.137031     0.978938              0.0    11.607584   
...            ...       ...          ...              ...          ...   
24999995         0 -0.010000     1.000000              0.0 -9999.000000   
24999996         0 -0.010000     1.000000              0.0 -9999.000000   
24999997         0 -0.010000     1.000000              0.0 -9999.000000   
24999998         0 -0.010000     1.000000              0.0 -9999.000000   
24999999         0 -0.010000     1.000000              0.0 -9999.000000   

               DEM_raw  DEM_ditch_detection  DEM_ditch_detection_streams  \
0           134.899994 

2025-03-30 13:35:22,753 - distributed.core - INFO - Connection to tcp://127.0.0.1:36645 has been closed.
2025-03-30 13:35:22,753 - distributed.core - INFO - Connection to tcp://127.0.0.1:36645 has been closed.
2025-03-30 13:35:23,142 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.


Added column: skyview_median_2
Applied mask to the array.
Applied mask for skyview_median_2 with radius 2
Generating feature: skyview_median_4
Added column: skyview_median_4
Applied mask to the array.
Applied mask for skyview_median_4 with radius 4
Generating feature: skyview_median_6
Added column: skyview_median_6
Applied mask to the array.
Applied mask for skyview_median_6 with radius 6
Generating feature: skyview_std_6
Added column: skyview_std_6
Applied mask to the array.
Applied mask for skyview_std_6 with radius 6
Generating feature: skyview_skew_2
Added column: skyview_skew_2
Applied mask to the array.
Applied mask for skyview_skew_2 with radius 2
Generating feature: impoundment_mean_3
Added column: impoundment_mean_3
Applied mask to the array.
Applied mask for impoundment_mean_3 with radius 3
Generating feature: impoundment_mean_2
Added column: impoundment_mean_2
Applied mask to the array.
Applied mask for impoundment_mean_2 with radius 2
Generating feature: impoundment_mean_6


### Closing Dask workers

In [7]:
from dask.distributed import LocalCluster, Client

# Create a cluster
cluster = LocalCluster()

# Connect to the Dask cluster (if not already connected)
client = Client(cluster)

# Close the client and cluster
client.close()  # Closes the client
cluster.close()  # Shuts down the cluster completely

# Optionally check if all workers are stopped
print("Cluster workers:", cluster.scheduler_info['workers'])

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42341 instead
  warnings.warn(


Cluster workers: {}


### View the test pickle
StreamAmp result viewing. Should the treshold be tested?

In [8]:
import pickle

# Load the pickle file
pickle_file = '../../01_Data/01_Raw/features/zone_1.pickle'  # Path to the pickle file
with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

# Print the keys of the data to inspect its structure
print("Available keys in the pickle file:", data.keys())

# If the data is a large structure, you may also want to inspect a sample of the values
for key in data.keys():
    print(f"Sample data for key '{key}':", data[key])


Available keys in the pickle file: Index(['label_3m', 'hpmf_raw', 'skyview_raw', 'impoundment_raw', 'slope_raw',
       'DEM_ditch_detection', 'DEM_ditch_detection_streams', 'conic_mean',
       'skyview_gabor', 'skyview_min_6', 'skyview_median_2',
       'skyview_median_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6',
       'skyview_std_6', 'impoundment_amplified', 'impoundment_mean_2',
       'impoundment_mean_3', 'impoundment_mean_6', 'impoundment_median_2',
       'impoundment_median_4', 'impoundment_median_6', 'impoundment_max_6',
       'impoundment_std_4', 'impoundment_std_6', 'hpmf_f',
       'hpmf_f_visualisation', 'hpmf_min_4', 'hpmf_mean_3', 'hpmf_mean_6',
       'hpmf_mean_4', 'hpmf_median_4', 'hpmf_std_6', 'slope_channels',
       'slope_min_2', 'slope_min_4', 'slope_min_6', 'slope_median_6',
       'slope_std_4', 'slope_std_6'],
      dtype='object')
Sample data for key 'label_3m': 0           0
1           0
2           0
3           0
4           0
           ..


In [9]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load the pickle file
pickle_file = '../../01_Data/01_Raw/features/zone_6.pickle'  # Path to the pickle file
with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

# Step 2: Inspect the loaded data
if isinstance(data, np.ndarray):  # If it's an array
    print("Loaded data is an ndarray with shape:", data.shape)
    print("Sample values:", data)

    # Visualize as a raster plot
    plt.figure(figsize=(10, 8))
    plt.imshow(data, cmap='Blues', interpolation='nearest')
    plt.colorbar(label='Values')
    plt.title('Raster Plot of Zone 1 Data')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.show()

elif isinstance(data, list):  # If it's a list
    print("Loaded data is a list with length:", len(data))
    print("Sample first entry:", data[0])

    # If the first entry is an array, visualize it
    if isinstance(data[0], np.ndarray):
        plt.figure(figsize=(10, 8))
        plt.imshow(data[0], cmap='Blues', interpolation='nearest')
        plt.colorbar(label='Values')
        plt.title('Raster Plot of First Array in List')
        plt.xlabel('X-coordinate')
        plt.ylabel('Y-coordinate')
        plt.show()
elif isinstance(data, dict):  # If it's a dictionary
    print("Loaded data is a dictionary with keys:", data.keys())
    for key in data:
        print(f"Key '{key}' contains data of type {type(data[key])}")

else:  # If it's something else
    print("Loaded data type:", type(data))
    print("Content preview:", data)


Loaded data type: <class 'pandas.core.frame.DataFrame'>
Content preview:           label_3m  hpmf_raw  skyview_raw  impoundment_raw    slope_raw  \
0                0  0.022031     0.999796         0.000985     0.676565   
1                0  0.022344     0.999836         0.000328     0.822603   
2                0  0.017031     0.999846         0.003124     0.976215   
3                0  0.006094     0.999838         0.009373     1.058265   
4                0 -0.004844     0.999837         0.015622     1.043701   
...            ...       ...          ...              ...          ...   
24999995         0  0.045312     0.996912         0.000843 -9999.000000   
24999996         0  0.052187     0.996843         0.001405 -9999.000000   
24999997         0  0.059063     0.996729         0.001968 -9999.000000   
24999998         0  0.059375     0.996570         0.006468 -9999.000000   
24999999         0  0.053125     0.996369         0.014907 -9999.000000   

          DEM_ditch_detect

In [10]:
import os
print("File size:", os.path.getsize('../../01_Data/01_Raw/features/zone_5.pickle'), "bytes")


File size: 5850004061 bytes
